In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
# Converter caracteres para "0" (ou outra constante)
# substituir - por 0
#df.loc[df['attribute']=='-','attribute'] = '0'



In [ ]:
# Converter para inteiro (ou float) e caracteres para NaN - Not a Number

# coerce obriga a conversão, jogando para Nan que não foi convertido
#df.loc[:,'attribute'] = pd.to_numeric(df.loc[:,'attribute'], downcast='integer', errors='coerce')
#df.loc[:,'attribute'] = pd.to_numeric(df.loc[:,'attribute'], downcast='float', errors='coerce')


# Outra opção
#data.loc[data['attribute'] == 'P', 'Sex'] = np.nan

In [ ]:
dtrain_orig = dtrain.copy()

# remove duplicatas
dtrain_nd = dtrain.drop_duplicates(keep='first')
print("Dados originais: %d, após remocao duplicatas: %d" % (dtrain.shape[0], dtrain_nd.shape[0]))

# remove outliers
attributes = ['tax','total','insurance', 'bathroom']
dtrain_out = remove_outliers_IQR(dtrain_nd, attributes)
dtest_out = remove_outliers_IQR(dtest, attributes)

In [ ]:
def check_missing_att(df):
    # verificando atributos faltantes
    for var in df:
        # somamos o número de variáveis nulas (NaN)
        nullatt = np.sum(df[var].isnull())
        if nullatt > 0:
            print(var,"- faltantes: ", end='')
            print(nullatt)

In [ ]:
def fill_missing_average(df):
        
    dfn = df.copy()
    for i in dfn:
        nullatt = np.sum(dfn[i].isnull())
        if nullatt == 0: continue
        print(i, nullatt, 'faltantes')
        
        # obtem a media para os dados completos no atributo
        # dfn[i].notnull() -> elementos completos no atributo i
        # dfn.loc[dfn[i].notnull(),i] -> filtra os valores completos do atributo i
        avg_compl = dfn.loc[dfn[i].notnull(),i].mean().astype(dfn[i].dtype)
        print('\tmédia =', avg_compl)
        dfn.loc[dfn[i].isnull(),i] = avg_compl
        
    return dfn

In [ ]:
#check_missing_att(df)

In [ ]:
def fill_missing_regression(df, attributes, attributes_reg):
    
    """ Funcao para preencher dados faltantes por regressao
        parametros:
            df - dataframe
            attributes - lista de atributos a preencher (alvo/saida)
            attributes_reg - lista dos respectivos atributos a serem usados na regressao
                             como variaveis de entrada
    """
    
    dfn = df.copy()
    print(len(attributes), ' attributos: ')
    
    for i in range(len(attributes)):
        print('-', i, ': preencher ', attributes[i], ' usando ', attributes_reg[i], end=' -- ')
        nullatt = np.sum(dfn[attributes[i]].isnull())
        print(nullatt, ' faltantes')
        if nullatt == 0: continue
        
        # atributo de entrada para dados completos
        Xa = np.array(dfn.loc[dfn[attributes[i]].notnull() & dfn[attributes_reg[i]].notnull(), 
                              attributes_reg[i]])
        
        # atributo de saida para dados completos
        Ya = np.array(dfn.loc[dfn[attributes[i]].notnull() & dfn[attributes_reg[i]].notnull(), 
                              attributes[i]])
        
        reg = LinearRegression()
        reg.fit(Xa.reshape(-1, 1), Ya.reshape(-1, 1))
        
        # seleciona as variáveis de entrada 
        # relativa aos valores faltantes para fazer a regressao
        Zr = dfn.loc[ dfn[attributes[i]].isnull(), attributes_reg[i]]
        
        # prediz as variaveis faltantes
        Za = reg.predict( (np.array(Zr)).reshape(-1, 1) )

        # monta array para substituir a variavel agora preenchida
        att_fill = np.array(dfn.loc[:, attributes[i]])
        att_fill[np.where(np.isnan(att_fill))] = np.array(Za.flatten()).astype(int)
        dfn.loc[:,attributes[i]] = att_fill.astype(np.int32)
        #print(dfn.loc[df[attributes[i]].isnull(), attributes[i]])
        
    return dfn

In [ ]:
# attributes     = ['rooms', 'parking spaces', 'area', 'tax']
# attributes_reg = ['area', 'rent', 'tax', 'hoa']

# dtrain_misr = fill_missing_regression(dtrain_out, attributes, attributes_reg)

# check_missing_att(dtrain_misr)

In [ ]:
# dtrain_mism = fill_missing_average(dtrain_out)
# check_missing_att(dtrain_mism)
# dtrain_mism.sample(5)